# Text Summarization

Alice continues her journey and now she is in 2015. Now it has become easier, as you can use word2vec! This time Alice needs help to solve the problem of summarizing news texts.

The task of summarization is to obtain a shorter text from the original text, which will contain all (or almost all) the information that was in the original text. Thus, from the text you need to obtain its summary in such a way as to lose as little information as possible.

Methods for solving this problem are usually divided into two categories:
- Extractive Summarization $-$ algorithms based on identifying the most informative parts of the source text (sentences, paragraphs, etc.) and compiling a summary from them.
- Abstractive Summarization $-$ algorithms that generate new text based on the source.

We will work with Extractive Summarization.

## 0. Dataset Preprocessing

In [52]:
import os
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from scipy import sparse
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading dataset

We will use data from the CNN/DailyMail news corpus.

In [2]:
DATA_DIR = './cnn_stories_short/'

In [3]:
%%capture

!wget https://www.dropbox.com/s/kofxrgod7kl720m/cnn_stories_short.zip
!mkdir cnn_data
!unzip cnn_stories_short.zip -d $DATA_DIR
!rm -r ./cnn_stories_short/__MACOSX

### Dataset preparation

The dataset consists of source texts and already written summaries for them. We will save original texts.

In [4]:
texts = []
for filename in os.listdir(DATA_DIR):
    with open(os.path.join(DATA_DIR,filename),'r') as input_file:
        all_texts = input_file.read().split('@highlight')
        texts.append(all_texts[0])

#### We will need:
* texts broken into sentences
* sentences broken into tokens
* texts, broken sentences that are broken into tokens

In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/xbar19/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
sent_tokenized_texts = [sent_tokenize(text) for text in texts]
tokenized_sentences = [word_tokenize(sent) for text in texts for sent in sent_tokenize(text)]
tokenized_texts = [[word_tokenize(sent) for sent in text] for text in sent_tokenized_texts]

### Loading Word Embedding Model

For the TextRank algorithm, we need to obtain a vector representation for each sentence in the text.

We will use pre-trained Glove vectors. **GloVe** (Global Vectors for Word Representation) is an unsupervised learning algorithm for obtaining vector representations for words, developed by Stanford University. It leverages global word-word co-occurrence statistics from a corpus to create dense vector embeddings that capture semantic meanings. GloVe vectors enable improved performance in various natural language processing tasks by representing words in a continuous vector space, where similar words are located closer together.

Let's load models:

The downloaded archive contains a set of files with vectors of different lengths. Each file stores a word on each line, followed by a space, the values ​​of the vector representation of this word.

In [28]:
word_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f.readlines():
        values = line.split()
        word = values[0]
        word_embeddings[word] = np.asarray(values[1:], dtype='float32')

We stored vectors to word_embeddings value. Thus, word_embeddings is a dictionary, where key is a word and value is a vector of this word.

## Task 1: Word2Vec text representation

*   For each text obtaint it's vector representation by averaging word2vec representation of each word. Just sum it component by component and divide on number of words in sentence. If word embedding model do not contain word initialize it with zeros. Use word representations saved in word_embeddings values.
*   Count cosine similarity between each sentences and obtain matrix of cosine similarity **G**.

In [90]:
# TODO complete transform function. You can add additional values in class constructor if neccesary.

class TfidfEmbeddingVectorizer:

    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model # word embedding model
        self.dim = dim

    def transform(self, X):
        # Take list of texts as input, returns np.array with vector representation of each text.
        sentence_vectors = []
        for sentence in X:
            word_vectors = []
            for word in sentence:
                if word in self.embedding_model:
                    word_vectors.append(self.embedding_model[word])
                else:
                    word_vectors.append(np.zeros(self.dim))
            if len(word_vectors) > 0:
                sentence_vectors.append(np.mean(word_vectors, axis=0))
            else:
                sentence_vectors.append(np.zeros(self.dim))
        return np.array(sentence_vectors)

In [91]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)

### Building the Cosine Similarity Matrix

For the *TextRank* algorithm, we need to build a weighted graph from the text. The graph will be represented as a matrix of cosine similarity between sentences.

For example, let's build a graph in the form of a distance matrix for one of the texts.
Let's choose one text and build a distance matrix for it. We'll use the cosine distance as a metric.

In [92]:
TEXT_NUM = 5

In [93]:
sentences = tokenized_texts[TEXT_NUM]

Using the vectorizer, we will obtain vectors for all sentences of the text.

In [94]:
vectorized_sentences = sentence_vectorizer.transform(sentences)

Let's calculate the matrix with cosine distances.

In [95]:
## TODO calculate the matrix of cosine similarity and assign it to G value.

def get_cosine_similarity_matrix(sentences):
    similarity_matrix = cosine_similarity(sentences)
    return similarity_matrix

G = get_cosine_similarity_matrix(vectorized_sentences)

## Extractive Summarization $-$ TextRank

Now we will implement the text summarization method itself. It will be based on the *PageRank* algorithm.

*PageRank* $-$ is a recursive algorithm that evaluates the importance of each node in the graph based on its connections with other nodes. Initially, the algorithm was used to evaluate the importance of Internet pages for search engines.

The adaptation of this algorithm for text summarization is called *TextRank*.

The algorithm sequentially goes through all the nodes in the graph and recalculates the PageRank values ​​for each of them using the formula below.

This happens until the process stabilizes, that is, the *PageRank* values ​​for all nodes stop changing significantly with each new iteration.

$$ G = (V,E) - граф $$
$$$$
$$ PageRank(v) = \frac{(1-d)}{N} +  d \sum_{u} \frac {PageRank(u) * W_{(u, v)}} {C(u)}$$

$$v\ -\ вершина\ графа, v \in V $$

$$u\ -\ вершины\ графа,\ такие\ что\ (u,v) \in E$$

$$C(u) - количество \ вершин, \ таких \ что (u,v) \in E$$

$$W_{(u, v)} - вес\ ребра\ (u, v) \in E $$

$$d = 0,85\ -\ коэффициент\ затухания$$

Let's use NetworkX library to Page Rank algorithm.

In [99]:
!pip install networkx


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [100]:
import networkx as nx

nx_graph = nx.from_numpy_array(G)
nx_scores = nx.pagerank(nx_graph)

In [101]:
ranked_sentences = sorted(((nx_scores[i], s, i) for i,s in enumerate(sentences)), reverse=True)

Let's output 5 sentences with the highest TextRank. This will be our final text summation.

In [106]:
SUMMARY_LEN = 5

for i in range(SUMMARY_LEN):
    print(' '.join(ranked_sentences[i][1]))

For the next 20 minutes , Tuominen and information security researcher Robert Lee gave me a tutorial on making my iPhone settings hacker-proof .
For more tips , check out the video .
( CNNMoney ) -- Here 's one way to make your iPhone hacker-proof : Ask hackers for advice .
Others take a bit more work , such as using settings to limit ad tracking .
Some of them are basic -- like changing your four character password to something more complex .


Now let's combine everything into one summarize function, which will receive text divided into sentences as input and output 5 sentences with the highest *TextRank*.

In [107]:
def summarize(sentences,summary_len=5):
    vectorized_sentences = sentence_vectorizer.transform(sentences)
    G = get_cosine_similarity_matrix(vectorized_sentences)
    nx_graph = nx.from_numpy_array(G)
    nx_scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((nx_scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)
    summary = []
    for i in range(summary_len):
        summary.append(' '.join(ranked_sentences[i][1]))
    return summary

In [109]:
summarize(tokenized_texts[5])

['For the next 20 minutes , Tuominen and information security researcher Robert Lee gave me a tutorial on making my iPhone settings hacker-proof .',
 'For more tips , check out the video .',
 "( CNNMoney ) -- Here 's one way to make your iPhone hacker-proof : Ask hackers for advice .",
 'Others take a bit more work , such as using settings to limit ad tracking .',
 'Some of them are basic -- like changing your four character password to something more complex .']

Let's get summaries for all our texts:

In [116]:
system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]

/tmp/ipykernel_45739/557689583.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]


  0%|          | 0/300 [00:00<?, ?it/s]

Let's look on the 10th sample

In [117]:
print("\n".join(system_summaries[10][:5]))

Another high-profile striker on the move Monday was Tottenham 's Robbie Keane , who has agreed a loan switch to London rivals West Ham until the end of this season .
Villa allowed Irish midfielder Stephen Ireland to join Newcastle on loan for the rest of this season , almost six months after he arrived from Manchester City .
( CNN ) -- Fernando Torres ' highly-anticipated switch to English champions Chelsea was completed just before Monday transfer window closed in a deal that his former team Liverpool said had broken the British record .
Torres ' move followed the news that Liverpool had agreed a club-record fee for Newcastle 's England striker Andy Carroll , who handed in a transfer request in the afternoon after the 18-time English champions made an improved bid .
Stoke also allowed former Chelsea and Barcelona striker Eidur Gudjohnsen to join English rivals Fulham on loan , with the Iceland international having failed to win a regular place since his pre-season move from Monaco .


## Task 2 IDF word2vec modification

Modify your previous solution. For each text obtaint it's vector representation by averaging word2vec representation of each word multiplied by the IDF value of this word.

In [151]:
 #TODO complete transform function. You can add additional values in class constructor if neccesary.


class TfidfEmbeddingVectorizer:

    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model
        self.dim = dim
        self.vectorizer = TfidfVectorizer()
        self.vocab = None
        self.idf = None

    def fit(self, X):
        # TF-IDF fitting
        self.vectorizer.fit([' '.join(doc) for doc in X])
        self.vocab = self.vectorizer.vocabulary_
        self.idf = self.vectorizer.idf_
        return self

    def transform(self, X):
        # Take list of texts as input, returns np.array with vector representation of each text.
        sentence_vectors = []

        for doc in X:
            word_vectors = []
            total_weight = 0.0

            for word in doc:
                if word in self.embedding_model:
                    if word in self.vocab:
                        word_id = self.vocab[word]
                        idf_value = self.idf[word_id]
                    else:
                        idf_value = 1.0 

                    word_vector = self.embedding_model[word] * idf_value
                    word_vectors.append(word_vector)
                    total_weight += idf_value
                else:
                    word_vectors.append(np.zeros(self.dim))

            if len(word_vectors) > 0:
                sentence_vector = np.sum(word_vectors, axis=0) / total_weight
            else:
                sentence_vector = np.zeros(self.dim)

            sentence_vectors.append(sentence_vector)

        return np.array(sentence_vectors)

In [152]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)
sentence_vectorizer = sentence_vectorizer.fit(tokenized_sentences)

In [153]:
## TODO copy your function for cosine similarity here

def get_cosine_similarity_matrix(sentences):
    similarity_matrix = cosine_similarity(sentences)
    return similarity_matrix

In [154]:
def summarize(sentences,summary_len=5):
    vectorized_sentences = sentence_vectorizer.transform(sentences)
    G = get_cosine_similarity_matrix(vectorized_sentences)
    nx_graph = nx.from_numpy_array(G)
    nx_scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((nx_scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)
    summary = []
    for i in range(summary_len):
        summary.append(' '.join(ranked_sentences[i][1]))
    return summary

 Summarize your texts

In [155]:
system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]

/tmp/ipykernel_45739/557689583.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]


  0%|          | 0/300 [00:00<?, ?it/s]

Print summary for 7-th sample:

In [157]:
print("\n".join(system_summaries[7]))

`` Any attempt to rescue the girls by force will end in a tragic result -- it could be Nigeria 's Beslan , '' he said , referring to the attempt to free hostages at a school in Russia in 2004 that left more than 300 dead , many of them children .
Nor would Zenn be surprised if Shekau were under the influence of drugs when he recorded his video statement ; they have sometimes been found in Boko Haram camps by the Nigerian military .
`` Boko Haram has likely split up or sold the girls into many small groups , '' and they can be used as human shields in the event of an attack , he said .
Sani believes Boko Haram targeted the girls to force concessions from the Nigerian government -- beginning perhaps with the release of Boko Haram followers from prisons .
In his video , Shekau said of the girls : `` We would also give their hands in marriage because they are our slaves . ''
